In [15]:
# Importar el archivo EDA para obtener el DataFrame procesado

import import_ipynb
import eda # Esto importa todo el contenido de eda.ipynb

# acceder al DataFrame preprocesado (df) desde eda.ipynb
df = eda.df




#### Preparación de los datos

Este paso prepara los datos para el modelado, separando las características ( X) y la variable objetivo ( y), dividiendo en entrenamiento y prueba, y normalizando los datos.

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import optuna


# Separar las características y la variable objetivo
X = df.drop('custcat', axis=1)
y = df['custcat']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalización para todas las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Función para optimizar modelos

Utilizamos Optuna para encontrar los mejores hiperparámetros de cada modelo.

In [17]:
def optimize_model(model, param_space):
    def objective(trial):
        # Genera los parámetros para el modelo
        params = {k: v(trial) for k, v in param_space.items()}
        # Construye un pipeline con escalador y modelo
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model(**params))
        ])
        # Realiza validación cruzada y calcula el promedio de accuracy
        return cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)
    return study.best_params


#### Función para evaluar modelos
Evalúa el rendimiento en el conjunto de entrenamiento y prueba. Imprime el precision y el overfitting (diferencia entre precision de entrenamiento y prueba).

In [18]:
def evaluate_model(model, params, name):
    # Construir pipeline con escalador y modelo
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model(**params))
    ])
    # Entrenar el modelo
    pipeline.fit(X_train, y_train)
    # Evaluar en entrenamiento y prueba
    train_accuracy = pipeline.score(X_train, y_train)
    test_accuracy = pipeline.score(X_test, y_test)
    overfitting = train_accuracy - test_accuracy
    
    # Predicciones en el conjunto de prueba
    y_pred = pipeline.predict(X_test)
    
    # Imprimir métricas
    print(f"\nResultados para {name}:")
    print(f"Accuracy en entrenamiento: {train_accuracy:.4f}")
    print(f"Accuracy en prueba: {test_accuracy:.4f}")
    print(f"Diferencia (Overfitting): {overfitting:.4f}")
    print("\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred))
    
    return pipeline


#### Modelos y optimización


In [19]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier

rf_param_space = {
    'n_estimators': lambda trial: trial.suggest_int('n_estimators', 50, 200),
    'max_depth': lambda trial: trial.suggest_int('max_depth', 2, 20),
    'min_samples_split': lambda trial: trial.suggest_int('min_samples_split', 2, 10),
    'min_samples_leaf': lambda trial: trial.suggest_int('min_samples_leaf', 1, 5)
}
rf_best_params = optimize_model(RandomForestClassifier, rf_param_space)
rf_pipeline = evaluate_model(RandomForestClassifier, rf_best_params, "Random Forest")


[I 2024-12-09 16:47:42,468] A new study created in memory with name: no-name-223629c1-2baf-4a35-be4a-a3b8456b98a1
[I 2024-12-09 16:47:43,543] Trial 0 finished with value: 0.35625 and parameters: {'n_estimators': 85, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.35625.
[I 2024-12-09 16:47:44,225] Trial 1 finished with value: 0.39875 and parameters: {'n_estimators': 72, 'max_depth': 6, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.39875.
[I 2024-12-09 16:47:45,046] Trial 2 finished with value: 0.37749999999999995 and parameters: {'n_estimators': 74, 'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.39875.
[I 2024-12-09 16:47:46,944] Trial 3 finished with value: 0.38999999999999996 and parameters: {'n_estimators': 187, 'max_depth': 16, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.39875.
[I 2024-12-09 16:47:48,346] Trial 4 finished w


Resultados para Random Forest:
Accuracy en entrenamiento: 0.4188
Accuracy en prueba: 0.3950
Diferencia (Overfitting): 0.0237

Reporte de Clasificación:
              precision    recall  f1-score   support

           1       0.40      0.55      0.46        53
           2       0.30      0.07      0.11        44
           3       0.38      0.55      0.45        56
           4       0.44      0.34      0.39        47

    accuracy                           0.40       200
   macro avg       0.38      0.38      0.35       200
weighted avg       0.38      0.40      0.36       200



In [20]:
#Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier

gb_param_space = {
    'n_estimators': lambda trial: trial.suggest_int('n_estimators', 50, 200),
    'learning_rate': lambda trial: trial.suggest_loguniform('learning_rate', 0.01, 0.1),
    'max_depth': lambda trial: trial.suggest_int('max_depth', 2, 10)
}
gb_best_params = optimize_model(GradientBoostingClassifier, gb_param_space)
gb_pipeline = evaluate_model(GradientBoostingClassifier, gb_best_params, "Gradient Boosting")


[I 2024-12-09 16:48:25,736] A new study created in memory with name: no-name-0596e1bf-27ab-4fc1-b8d4-745f0f928008
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\3716008389.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': lambda trial: trial.suggest_loguniform('learning_rate', 0.01, 0.1),
[I 2024-12-09 16:49:18,068] Trial 0 finished with value: 0.33499999999999996 and parameters: {'n_estimators': 140, 'learning_rate': 0.09422876746184224, 'max_depth': 10}. Best is trial 0 with value: 0.33499999999999996.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\3716008389.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': 


Resultados para Gradient Boosting:
Accuracy en entrenamiento: 0.4900
Accuracy en prueba: 0.4150
Diferencia (Overfitting): 0.0750

Reporte de Clasificación:
              precision    recall  f1-score   support

           1       0.40      0.42      0.41        53
           2       0.38      0.27      0.32        44
           3       0.43      0.50      0.46        56
           4       0.44      0.45      0.44        47

    accuracy                           0.41       200
   macro avg       0.41      0.41      0.41       200
weighted avg       0.41      0.41      0.41       200



In [10]:
# Red Neuronal
'''
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import optuna

# Parametros para la red neuronal
nn_param_space = {
    'hidden_layer_sizes': lambda trial: (
        trial.suggest_int('n_units_l1', 50, 200),  # Primera capa entre 50 y 200 neuronas
        trial.suggest_int('n_units_l2', 25, 100)   # Segunda capa entre 25 y 100 neuronas
    ),
    'alpha': lambda trial: trial.suggest_loguniform('alpha', 1e-5, 1e-1),
    'learning_rate_init': lambda trial: trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)
}

# Función de optimización con Optuna
def optimize_model(model, param_space, X_train, y_train):
    def objective(trial):
        # Obtiene los parámetros del espacio de búsqueda
        params = {k: v(trial) for k, v in param_space.items()}
        
        # Asegúrate de que los parámetros se devuelvan correctamente
        print(f"Optimized parameters: {params}")
        
        # Realizar validación cruzada con 5 pliegues
        return cross_val_score(Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model(**params))
        ]), X_train, y_train, cv=5, scoring='accuracy').mean()

    # Crear el estudio de Optuna y optimizar
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)
    
    # Asegurarse de que los mejores parámetros se devuelvan correctamente
    best_params = study.best_params
    print(f"Best parameters found: {best_params}")
    return best_params

# Evaluar el modelo con los mejores parámetros obtenidos
nn_best_params = optimize_model(MLPClassifier, nn_param_space, X_train, y_train)

# Comprobar los parámetros antes de pasarlos al modelo
print("Best Parameters for MLP: ", nn_best_params)

# Construir el pipeline con los mejores parámetros
hidden_layer_sizes = nn_best_params['hidden_layer_sizes']
print("Hidden layer sizes:", hidden_layer_sizes)

# Verificar que los parámetros están correctamente formateados como tupla
if isinstance(hidden_layer_sizes, tuple):
    nn_model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        alpha=nn_best_params['alpha'],
        learning_rate_init=nn_best_params['learning_rate_init']
    )
    # Evaluar el modelo
    evaluate_model(nn_model, nn_best_params, "Red Neuronal")
else:
    print("Error: 'hidden_layer_sizes' debe ser una tupla.")

'''


[I 2024-12-09 16:37:49,591] A new study created in memory with name: no-name-0407094d-a0d3-457f-9eb8-3010b4037eea
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\1653340466.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': lambda trial: trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\1653340466.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate_init': lambda trial: trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)


Optimized parameters: {'hidden_layer_sizes': (86, 91), 'alpha': 0.0002863810830491898, 'learning_rate_init': 0.0028651921813825673}


c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: Con

Optimized parameters: {'hidden_layer_sizes': (89, 93), 'alpha': 0.0036844120314429556, 'learning_rate_init': 0.037706075352101326}


[I 2024-12-09 16:37:57,140] Trial 1 finished with value: 0.3075 and parameters: {'n_units_l1': 89, 'n_units_l2': 93, 'alpha': 0.0036844120314429556, 'learning_rate_init': 0.037706075352101326}. Best is trial 0 with value: 0.34125.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\1653340466.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': lambda trial: trial.suggest_loguniform('alpha', 1e-5, 1e-1),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\1653340466.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate_init': lambda trial: trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)


...

c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: Con

Optimized parameters: {'hidden_layer_sizes': (118, 28), 'alpha': 0.005611330123813928, 'learning_rate_init': 0.0004658411686775387}


c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: Con

Best parameters found: {'n_units_l1': 92, 'n_units_l2': 39, 'alpha': 0.005343344453576201, 'learning_rate_init': 0.00014431163507430748}
Best Parameters for MLP:  {'n_units_l1': 92, 'n_units_l2': 39, 'alpha': 0.005343344453576201, 'learning_rate_init': 0.00014431163507430748}


KeyError: 'hidden_layer_sizes'

In [26]:
#Regresion Logistica Multiclase

from sklearn.linear_model import LogisticRegression

lr_param_space = {
    'C': lambda trial: trial.suggest_loguniform('C', 1e-5, 1e5),
    'solver': lambda trial: trial.suggest_categorical('solver', ['lbfgs', 'newton-cg', 'sag', 'saga']),
    'multi_class': lambda trial: 'multinomial'
}
lr_best_params = optimize_model(LogisticRegression, lr_param_space)
lr_pipeline = evaluate_model(LogisticRegression, lr_best_params, "Regresión Logística Multiclase")


[I 2024-12-09 17:02:53,161] A new study created in memory with name: no-name-c18a0660-3dc6-442f-9a56-1100d8a5e167
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\1993323069.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': lambda trial: trial.suggest_loguniform('C', 1e-5, 1e5),
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\Administrator\Desktop\Proyecto_ML Refuerzo\ML_Refuerzo_Esther\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be


Resultados para Regresión Logística Multiclase:
Accuracy en entrenamiento: 0.4163
Accuracy en prueba: 0.4050
Diferencia (Overfitting): 0.0112

Reporte de Clasificación:
              precision    recall  f1-score   support

           1       0.38      0.43      0.41        53
           2       0.33      0.11      0.17        44
           3       0.41      0.55      0.47        56
           4       0.45      0.47      0.46        47

    accuracy                           0.41       200
   macro avg       0.39      0.39      0.38       200
weighted avg       0.39      0.41      0.38       200



In [22]:
#SVM Multiclase

from sklearn.svm import SVC

svm_param_space = {
    'C': lambda trial: trial.suggest_loguniform('C', 1e-2, 10),
    'gamma': lambda trial: trial.suggest_loguniform('gamma', 1e-4, 1e-1),
    'kernel': lambda trial: trial.suggest_categorical('kernel', ['linear', 'rbf'])
}
svm_best_params = optimize_model(SVC, svm_param_space)
svm_pipeline = evaluate_model(SVC, svm_best_params, "SVM Multiclase")


[I 2024-12-09 16:55:40,668] A new study created in memory with name: no-name-3b6c3ddf-1efc-4585-8bb9-baa86a7a39fe
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\3190910586.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': lambda trial: trial.suggest_loguniform('C', 1e-2, 10),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14812\3190910586.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': lambda trial: trial.suggest_loguniform('gamma', 1e-4, 1e-1),
[I 2024-12-09 16:55:40,810] Trial 0 finished with value: 0.38625 and parameters: {'C': 0.05267734387988093, 'gamma': 0.0007667073414134454, 'kernel': 'linear'}. Best is trial 0 with value: 0.3


Resultados para SVM Multiclase:
Accuracy en entrenamiento: 0.5162
Accuracy en prueba: 0.3600
Diferencia (Overfitting): 0.1562

Reporte de Clasificación:
              precision    recall  f1-score   support

           1       0.35      0.36      0.35        53
           2       0.28      0.11      0.16        44
           3       0.38      0.50      0.43        56
           4       0.38      0.43      0.40        47

    accuracy                           0.36       200
   macro avg       0.34      0.35      0.34       200
weighted avg       0.35      0.36      0.34       200



#### Conjunto final
Creamos un conjunto de modelos usando VotingClassifier .

In [ ]:
from sklearn.svm import SVC

# Asegúrate de habilitar la opción 'probability=True' en el SVC
svc_model = SVC(probability=True)

# Agregar el SVC con probability=True al pipeline o al ensemble


In [28]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

# Crear el pipeline para SVM con probability=True
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC(probability=True))  # Asegurarse de que 'probability=True' esté activado
])

# Crear el ensemble
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf_pipeline['classifier']),
        ('gb', gb_pipeline['classifier']),
        #('nn', nn_pipeline['classifier']),
        ('svm', svm_pipeline['classifier']),
        ('lr', lr_pipeline['classifier'])
    ],
    voting='soft'
)
ensemble.fit(X_train, y_train)

# Evaluación del ensemble
train_accuracy = ensemble.score(X_train, y_train)
test_accuracy = ensemble.score(X_test, y_test)
overfitting = train_accuracy - test_accuracy
y_pred = ensemble.predict(X_test)

print("\nResultados para el Ensemble:")
print(f"Accuracy en entrenamiento: {train_accuracy:.4f}")
print(f"Accuracy en prueba: {test_accuracy:.4f}")
print(f"Diferencia (Overfitting): {overfitting:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))



Resultados para el Ensemble:
Accuracy en entrenamiento: 0.4650
Accuracy en prueba: 0.3700
Diferencia (Overfitting): 0.0950

Reporte de Clasificación:
              precision    recall  f1-score   support

           1       0.38      0.45      0.41        53
           2       0.22      0.09      0.13        44
           3       0.40      0.50      0.44        56
           4       0.37      0.38      0.38        47

    accuracy                           0.37       200
   macro avg       0.34      0.36      0.34       200
weighted avg       0.35      0.37      0.35       200



#### Guardar modelo

In [29]:
import joblib

# Guarda el modelo del ensemble
joblib.dump(ensemble, 'ensemble_model.pkl')


['ensemble_model.pkl']

In [30]:
from sklearn.preprocessing import StandardScaler


# Crear y ajustar el scaler
scaler = StandardScaler()
scaler.fit(X_train)

# Guardar el scaler en un archivo
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']